In [35]:
import scipy
import scipy.io as scio
from scipy import stats
from scipy import ndimage
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [2]:
def gaussian_filter_density(gt):
    print(gt.shape)
    density = np.zeros(gt.shape, dtype=np.float32)
    gt_count = np.count_nonzero(gt)
    if gt_count == 0:
        return density

    pts = np.array(list(zip(np.nonzero(gt)[1], np.nonzero(gt)[0])))
    #print("pts=",pts)
    leafsize = 2048
    # build kdtree
    #print 'build kdtree...'
    tree = scipy.spatial.KDTree(pts.copy(), leafsize=leafsize)
    # query kdtree
    #print 'query kdtree...' 
    distances, locations = tree.query(pts, k=4, eps=10.)

    #print 'generate density...'
    for i, pt in enumerate(pts):
        pt2d = np.zeros(gt.shape, dtype=np.float32)
        pt2d[pt[1],pt[0]] = 1.
        if gt_count > 1:
            sigma =((distances[i][1]+distances[i][2]+distances[i][3])/3)*0.3
        else:
            sigma = np.average(np.array(gt.shape))/2./2. #case: 1 point

        density += scipy.ndimage.filters.gaussian_filter(pt2d, sigma, mode='constant')
    #print 'done.'
    return density

In [2]:
def gaussian_filter_density(gt):
    print(gt.shape)
    density = np.zeros(gt.shape, dtype=np.float32)
    gt_count = np.count_nonzero(gt)
    if gt_count == 0:
        return density

    pts = np.array(list(zip(np.nonzero(gt)[1], np.nonzero(gt)[0])))
    #print("pts=",pts)
    leafsize = 2048
    # build kdtree
    #print 'build kdtree...'
    tree = scipy.spatial.KDTree(pts.copy(), leafsize=leafsize)
    # query kdtree
    #print 'query kdtree...' 
    distances, locations = tree.query(pts, k=4, eps=10.)

    #print 'generate density...'
    for i, pt in enumerate(pts):
        pt2d = np.zeros(gt.shape, dtype=np.float32)
        pt2d[pt[1],pt[0]] = 1.
        if gt_count > 1:
            sigma =((distances[i][1]+distances[i][2]+distances[i][3])/3)*0.3
        else:
            sigma = np.average(np.array(gt.shape))/2./2. #case: 1 point

        density += scipy.ndimage.filters.gaussian_filter(pt2d, sigma, mode='constant')
    #print 'done.'
    return density

In [3]:
"""for num in range(50,150):
    img = plt.imread('./ShanghaiTech/part_A/train_data/images/IMG_'+str(num)+'.jpg')
    mat = scio.loadmat('./ShanghaiTech/part_A/train_data/ground-truth/GT_IMG_'+str(num)+'.mat')
    k = np.zeros((img.shape[0],img.shape[1]))
    gt = mat["image_info"][0][0][0][0][0]
    for i in range(0,len(gt)):
        if int(gt[i][1])<img.shape[0] and int(gt[i][0])<img.shape[1]:
            k[int(gt[i][1]),int(gt[i][0])] = 1
    print("sum de k = ", np.sum(k))
    dens = gaussian_filter_density(k)
    np.save("./ShanghaiTech/part_A/train_data/GT_density_map/GT_density_IMG_" + str(num), dens)"""

'for num in range(50,150):\n    img = plt.imread(\'./ShanghaiTech/part_A/train_data/images/IMG_\'+str(num)+\'.jpg\')\n    mat = scio.loadmat(\'./ShanghaiTech/part_A/train_data/ground-truth/GT_IMG_\'+str(num)+\'.mat\')\n    k = np.zeros((img.shape[0],img.shape[1]))\n    gt = mat["image_info"][0][0][0][0][0]\n    for i in range(0,len(gt)):\n        if int(gt[i][1])<img.shape[0] and int(gt[i][0])<img.shape[1]:\n            k[int(gt[i][1]),int(gt[i][0])] = 1\n    print("sum de k = ", np.sum(k))\n    dens = gaussian_filter_density(k)\n    np.save("./ShanghaiTech/part_A/train_data/GT_density_map/GT_density_IMG_" + str(num), dens)'

In [4]:
#Counts total numvber of persons in a density map
"""counts = np.zeros(300)
for num in range(1,300):
    dens_map = np.load("ShanghaiTech/part_A/train_data/GT_density_map/GT_density_IMG_%s.npy"%str(num))
    counts[num-1] = np.sum(dens_map)
np.save("./ShanghaiTech/part_A/train_data/count_train_part_A", counts)"""  

'counts = np.zeros(300)\nfor num in range(1,300):\n    dens_map = np.load("ShanghaiTech/part_A/train_data/GT_density_map/GT_density_IMG_%s.npy"%str(num))\n    counts[num-1] = np.sum(dens_map)\nnp.save("./ShanghaiTech/part_A/train_data/count_train_part_A", counts)'

In [79]:
# SCRIPT PART FOR CROPPING IMAGES. Change paths accordingly.
images_path='./ShanghaiTech/part_A/train_data/images/'
densities_path="./ShanghaiTech/part_A/train_data/GT_density_map/"
#next are final paths to be used for getting training set
cropped_images_path= "./ShanghaiTech/part_A/train_data/images_cropped/" 
cropped_densities_path= "./ShanghaiTech/part_A/train_data/GT_density_map_cropped/"
counts=[0]*9*300
for num in range(1,301):
    if num%25==0 :
        print(num)
    img = cv2.imread(images_path+'IMG_'+str(num)+'.jpg',flags=cv2.IMREAD_COLOR)
    #img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    dens_map = np.load(densities_path+"GT_density_IMG_%s.npy"%str(num))
    h,w=img.shape[0],img.shape[1]
    #check ifi image has both dimensions greater than 224 otherwise skip
    if  w >= 224 and h >= 224:
        cropped_img=np.zeros((9,224,224,3)) 
        cropped_map=np.zeros((9,224,224)) # 9 density maps 
        if len(img.shape) < 3: # some images are in grey, ie only one canal
            cropped_img=np.zeros((9,224,224))
        
        # We make 4 crops at the 4 corners of image as well as the corresponding density map
        cropped_img[0], cropped_img[1]= img[:224,:224],img[:224,-224:]
        cropped_img[2],cropped_img[3]= img[-224:,:224], img[-224:,-224:]
        cropped_map[0],cropped_map[1]= dens_map[:224,:224], dens_map[:224,-224:]
        cropped_map[2],cropped_map[3]= dens_map[-224:,:224], dens_map[-224:,-224:]

        #We make five random crops of size 224x224 for both image and density map

        x=np.random.randint(0,w-224,5)
        y=np.random.randint(0,h-224,5)
        for i in range(4,9):
            cropped_img[i] = img[y[i-4]:y[i-4]+224,x[i-4]:x[i-4]+224]
            cropped_map[i] = dens_map[y[i-4]:y[i-4]+224,x[i-4]:x[i-4]+224]
        for i in range(9):
            cv2.imwrite(cropped_images_path+"cropped_IMG_"+ str(num)+"_"+str(i)+".jpg"
                        ,cropped_img[i])
            np.save(cropped_densities_path+"GT_density_cropped_IMG_" 
                    + str(num)+"_"+str(i), cropped_map[i])
            counts[i+num*(9-1)] = np.sum(cropped_map[i])
counts=np.asarray(counts)
np.save("./ShanghaiTech/part_A/train_data/count_train_part_A", counts)

25
50
75


KeyboardInterrupt: 